<a href="https://colab.research.google.com/github/lmoss/onesharp/blob/main/introOneSharp/move_copy_write.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic programs

We have seen the syntax of ```1#``` instructions 
[in a previous notebook](instructions.ipynb), along with the semantics
We turn to the simplest programs in the language.

In [ ]:
!python -m pip install -U setuptools
!python -m pip install -U git+https://github.com/lmoss/onesharp.git@main
from onesharp.interpreter.interpreter import *

## ```move_2_1```

The first is a program called ```move_2_1```.   It is designed to move the contents of R2 onto the end of R1, emptying out R2 in the process.  Written out in full it is 

```11#####111111###111###1##1111####1#111111####```

We have defined ```move_2_1``` to be this program, and so you can run it as shown below.

You can try the program by entering the following line.   In it


In [2]:
onesharp(move_2_1, ['11#','####1##11111'])

'11#####1##11111'

The words in the brackets are the contents of registers 1 and 2 when we start.  Again, our program moved the contents of R2 onto the end of R1, emptying R2.  You should want to change the opening contents of R1 and R2 in the cell above before going on.

(content:parsing)=
If you have an interpreter handy, you can parse this program and also get glosses.


In [3]:
parse(move_2_1)

['11#####', '111111###', '111###', '1##', '1111####', '1#', '111111####']

In [ ]:
parse_explain(move_2_1)

Here is this program $\moveprog_{2,1}$ expressed as a table.

<TABLE cellspacing="0" cellpadding="5">
        <TR BGCOLOR="#FFFFCC">
                <TD>11#####</TD> <TD></TD><TD style="text-aligh" right;>cases on R2</TD></TR>
        <TR BGCOLOR="#FFFFCC">
                <TD>111111###</TD> <TD></TD> <TD style="text-aligh" right;">R2 is empty; go forward 6 (we're done)
                  </TD>  </tr>
        <TR BGCOLOR="#FFFFCC"><TD>11111###</TD> <TD></TD><TD>go forward 3 to instruction 6 (the brown section)</TD>  
        </TR>
         <TR  BGCOLOR="#B0E0E6">
                <TD>1##</TD>  <TD></TD><TD>first symbol is #: add # to R1</TD></tr>
         <TR  BGCOLOR="#B0E0E6">
                <TD>1111####</TD> <TD></TD> <TD>go backward 4 to instruction 1 (the top)</TD></tr>   
         <TR  BGCOLOR="#D4B48C">
                <TD>1#</TD> <TD></TD> <TD>add 1 to R1</TD></tr>
         <TR  BGCOLOR="#D4B48C">
                <TD>111111####</TD> <TD></TD> <TD style="text-aligh" right;>go backward 6 to instruction 1 (the top) </TD>  
        </TR>
</TABLE> 

<br>


Here is an explanation of what is going on.

If R2 is empty, it goes to line 8.  Since the program itself only has 7 lines, this means that we have transferred *out* of the program.  We say that the program *halts* at that point.  (Please note that *halt* is an important technical term which we explain [in a different notebook](haltDef.ipynb).)


If the first symbol of R2 is a ```1```, then the second instruction after the case instruction at the top transfers us down to line 6. This part of the program would then add a ```1``` to R1 and return to the very beginning of the program.

If the first symbol of R2 is a ```#```, then we delete that # and go three steps forward, to line 4.  This part of the program would then add a ```#``` to R1 and return to the very beginning of the program.

The point is that by going around the loop repeatedly, we transfer the contents of R2 symbol-by-symbol to R1.
Similarly, whenever m and n are different numbers, we can build a program ```move_m_n```. This program would write the contents of Rm onto the end of Rn, emptying Rm in the process.





## Modifying ```move_2_1```

Suppose we want to modify ```move_2_1``` to get ```move_3_4```, a program which would copy the contents of R3 onto the end of R4 (and empty R4) in the process. Here is a way to do this which shows off some command-line tools that are part of the working environment that comes with the Python tools..

In [ ]:
parse(move_2_1)

When you enter the cell above, you get the program ```move_2_1``` as a Python *list* of instructions. We have seen the explanation of this parse above.  What we want to do in ```move_3_4``` is to change the overall "case" instruction in the beginning from ```11#####``` to ```111#####```.   And each time our program writes to a register, we want that register to be R4, not R1.  So we make two changes.

In [ ]:
unparse(pre_program)

We can check this out by entering it into the interpreter.  We could either copy the output line (without the quotes), and go up to the top of this notebook.  Alternatively, we could move the interpreter down to here using an up-arrow command that you will need to find.


```{exercise}
Write a program which takes the contents of R1 and adds them to the ends of *both* R2 and R3.
```

```{exercise}
Write a program that clears out R1, leaving it empty.
```

```{exercise}
Write a program that clears R3 and then swaps the contents of R1 and R2 (using the now-empty R3).
```


```{exercise}
Write a program $p$ that adds a $\one$ to the beginning of R1, assuming that R2 is empty. (For example, if R1 has $\hash\hash\one$ to start, then running $p$ would result in R1 having $\one\hash\hash\one$.) Of course, your program may use R2!
```

## ```copy```

The second program in this notebook is called ```copy```.
Like ```move```, the ```copy``` program is actually an infinite batch of programs. 

```{attention}
The difference between <i>move</i> and <i>copy</i>
for us is that when a register's contents are moved, the
register is left empty; but if the contents are copied,
then the register is left at the end with exactly what it had
at the beginning.
```

In order to copy in this way, we need an auxiliary register.
So while the ```move``` programs had two registers in their names, the ```copy``` programs have three.

<br><p>
Here is the idea behind copying Rm to Rn.   We use
an auxilliary register, say Rp.  We move Rm into Rn and Rp
at the same time, and then be move Rp back to Rm.
Of course, when we do this, we should have Rp empty to start
with.   
<p>

Here is our program when m = 1, n = 2, p = 3.  That is, here is $\copyprog_{1,2,3}$.

<TABLE cellspacing="0" cellpadding="5">
        <TR BGCOLOR="#FFFFCC">
                <TD>1#####</TD> <TD>cases on R1</TD></TR>
        <TR BGCOLOR="#FFFFCC">
                <TD>11111111###</TD> <TD>go forward 8 to instruction 10 (the  move<sub>3,1</sub> part) </TD>  </tr>
        <TR BGCOLOR="#FFFFCC"><TD>1111###</TD><TD>go forward 4 to instruction 7</TD>  
        </TR>
         <TR  BGCOLOR="#B0E0E6">
                <TD>11##</TD> <TD>add # to R2</TD></tr>
         <TR  BGCOLOR="#B0E0E6">
                <TD>111##</TD> <TD>add # to R3</TD></tr>   
         <TR  BGCOLOR="#B0E0E6">
                <TD>11111####</TD> <TD>go backward 5 to the top</TD> </TR>
         <TR  BGCOLOR="#D4B48C">
                <TD>11#</TD> <TD>add 1 to R2</TD></tr>
         <TR  BGCOLOR="#D4B48C">
                <TD>111#</TD> <TD>add 1 to R3</TD></tr>
         <TR  BGCOLOR="#D4B48C"> <TD>11111111####</TD> <TD>go backward 8 to the top</TD> </TR>
         <TR BGCOLOR="#FFFFCC"> <TD> move<sub>3,1</sub></TD><td> from earlier in this notebook </td></TR>
</TABLE>        

<br>

## ```write```

We continue with 
a program $\writeprog$ with 
the following properties:

1. When $\writeprog$ is started with $x$ in R1 
and R2 empty, we eventually halt  with a
word $y$
in R1 and all other registers empty.

2. Then $y$ is a program, and running
$y$ with R1 empty results in $x$ back in R1 and all
other registers empty.

In the notation from [a little later](functions), we have

$$
[\![ [\![ \writeprog]\!](x)]\!](\ ) = x
$$

Here is a table for $\writeprog$:

<TABLE  cellspacing="0" cellpadding="5">
        <TR BGCOLOR="#FFFFCC">
                <TD>1#####</TD> <TD>cases on R1</TD></TR>
        <TR BGCOLOR="#FFFFCC">
                <TD>111111111###</TD> <TD>go forward 9
                  to the move<sub>2,1</sub> part</TD>  </tr>
        <TR BGCOLOR="#FFFFCC"><TD>11111###</TD><TD>go forward 5:
to the brown part</TD>  
        </TR>
         <TR  BGCOLOR="#B0E0E6">
                <TD>11#</TD> <TD>add 1 to R2</TD></tr>
         <TR  BGCOLOR="#B0E0E6">
                <TD>11##</TD> <TD>add # to R2</TD></tr>   
         <TR  BGCOLOR="#B0E0E6">
                <TD>11##</TD> <TD>add # to R2</TD></tr>   
         <TR  BGCOLOR="#B0E0E6">
                <TD>111111####</TD> <TD>go backward 6 (to the top)</TD> </TR>
         <TR  BGCOLOR="#D4B48C">
                <TD>11#</TD> <TD>add 1 to R2 </TD></tr>
         <TR  BGCOLOR="#D4B48C">
                <TD>11##</TD> <TD>add # to R2</TD></tr>
         <TR  BGCOLOR="#D4B48C">
                <TD>111111111####</TD> <TD>go backward 9 (to the top)</TD>  
        </TR>
        <TR BGCOLOR="#FFFFCC">
             <TD> move<sub>2,1</sub></TD><td> from earlier in this notebook </td></TR>
</TABLE>        


<br>

It should be emphasized that $\writeprog$ reads from R1 
and writes to R2; it  makes no
use of any other register.

```{exercise}
Why is the result of running $\writeprog$ on a word $x$  always a program,
even if  $x$ is a word which is not a program?
```

```{exercise}
Modify $\writeprog$
 to get a program $\writetotwo$
with the following feature:
 If $\writetotwo$ is started with  $x$ in R1 
and R2 empty, we eventually halt  with a
word
$y$
in R2 and all other registers empty; moreover, running
$y$  with R2 empty results in $x$ back in R2  (not R1) and all
other registers empty.
```

# The $+$ notation on words

```{prf:definition}
If $x$ and $y$ are words, then $x + y$ is the *concatenation* of $x$ followed by $y$.  For example,

$$
\one\hash\one + \hash \hash = \one\hash\one  \hash \hash.
$$

This operation is associative: $x + (y+z) = (x+y) + z$.   

The *empty word* $\eps$ is an identity element for it:
$x + \eps = x = \eps + x$.
So we have a [*monoid*](https://en.wikipedia.org/wiki/Monoid)

$$
(\words, +, \eps)
$$
```

This means that we can "add" programs (since they are words).  For example, let 

$$
q =copyprog_{1,2,3} + \moveprog_{2,1}
$$

This program $q$ would take the contents of register 1, copy it into register 2 (using register 3), and then move register 2 back to register 1.  So assuming that registers 2 and 3 were empty to begin with, running $q$ with $x$ in register $1$ would give us $x + x$ in register 1 at the end.

```{exercise}
:label: pre-diag

1.  Let's take a word $x$ and run $\writeprog$ on it.  Call the resulting program $q$. Suppose that we run $q$ on a word $y$.  Which do we get: $x + y$ or $y + x$?  That is, which is true?

$$
[\![[\![\writeprog]\!](x)]\!](y) = x + y \quad \mbox{ or } \quad [\![[\![\writeprog]\!](x)]\!](y) = y + x
$$


2. Now let $p$ be the program ```copy_1_3_2 + write + move_3_1```. Find an equation satisfied by $p$. That is, fill in the blank

$$
[\![ p]\!](x) = \underline{\qquad\qquad\qquad}
$$

3.  Now fill in another blank, where $p$ again is ```copy_1_3_2 + write + move_3_1```:

$$
[\![ [\![ p]\!](x)]\!](y) = \underline{\qquad\qquad\qquad}
$$

```